In [ ]:
from ultralytics import YOLO
model = YOLO("Models/model_det_test/best.pt")

engine_file = model.export(format="engine", simplify=True, dynamic=True, imgsz=(640, 640), half=True, batch=20)

In [1]:
import ffmpegcv
from ultralytics import YOLO
import multiprocessing as mp
import torch
import torch.nn.functional as F
import time
import psutil
from pynvml import *
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np

In [7]:
model = YOLO('Models/model_det_test/best.engine', task='detect')

In [2]:
cap = ffmpegcv.toCUDA(ffmpegcv.VideoCaptureNV('video.mp4', pix_fmt='nv12'), tensor_format='chw')

In [3]:
ret, orig_frame = cap.read_torch()

In [ ]:
orig_frame.shape
torch.Size([3, 2160, 3840])

torch.Size([3, 2160, 3840])

In [ ]:
import torch

# Предположим, у тебя есть изображение orig_frame с shape [3, 2160, 3840]
# Добавим размер батча
input_tensor = orig_frame.unsqueeze(0)  # -> [1, 3, 2160, 3840]

# Убедимся, что тензор находится на том же устройстве, что и модель
input_tensor = input_tensor.float() / 255.0  # нормализация

# Запуск модели
with torch.no_grad():
    results = model(input_tensor, imgsz=640)

# Например, вывести результаты:
print(results)


ValueError: WARNING ⚠️ torch.Tensor inputs should be BCHW i.e. shape(1, 3, 640, 640) divisible by stride 32. Input shape(1, 3, 2160, 3840) is incompatible.

: 

In [ ]:
res[0].boxes[0].xyxy[0].cpu().numpy().astype(int)

array([   0,    0, 1700,  923])

: 

In [53]:
cap = ffmpegcv.toCUDA(ffmpegcv.VideoCaptureNV('video.mp4', pix_fmt='nv12'), tensor_format='chw')

In [50]:
inp = orig_frame.permute(1, 2, 0).cpu().numpy()
cv2_image = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)

In [42]:
inp.shape

(2160, 3840, 3)

In [43]:
n = cv2.imread('test1.jpg')

In [46]:
res = model.predict(n, conf=0.5, imgsz=640)


[04/16/2025-21:37:33] [TRT] [E] IExecutionContext::executeV2: Error Code 1: Cuda Runtime (invalid resource handle)
0: 384x640 300 potatos, 0.3ms
Speed: 2.7ms preprocess, 0.3ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)


In [49]:
res[0].boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [47]:
res[0].plot()

ValueError: cannot convert float NaN to integer

In [27]:
cv2.imwrite('test1.jpg', cv2_image)

[ WARN:0@634.524] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


True

In [10]:
inp.shape

(3, 2160, 3840)

In [5]:
orig_frame.shape

torch.Size([3, 2160, 3840])

In [12]:
frame_np = inp[:, :, ::-1]

In [14]:
model(frame_np)

error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/resize.cpp:3845: error: (-215:Assertion failed) !dsize.empty() in function 'resize'


In [ ]:
orig_frame

In [3]:
# Load model inside the process
model = YOLO('Models/model_det_test/best.engine', task='detect')

model_seg = YOLO('Models/model_seg_fp32/best_yoloseg.engine', task='segment')

In [ ]:
# Initialize CUDA context first
torch.cuda.init()
device = torch.device('cuda')

# Initialize GPU monitoring

# # Initialize video capture after CUDA init
cap = ffmpegcv.VideoCaptureNV('video.mp4', pix_fmt='rgb24')


out = ffmpegcv.VideoWriterNV('outpy1.mp4', 'h264', 25) 


area_for_segment = [1295, 1, 2338, 2154]
cm_per_pixel = 0.00999000999000999
camera_id = 1
colors = [144, 80, 70]

# === Tracking state ===
frame_count = 0
counter = 0
map_track_size = {}
track_set = set()

# Prepare a dictionary to store all potato data
potato_data = {}

# Performance monitoring setup
process = psutil.Process(os.getpid())
frame_count = 0
last_log_time = time.time()
start_time = last_log_time

while True:
    # Read frame
    ret, orig_frame = cap.read()
    if not ret:
        break

    # Preprocess
    with torch.no_grad():
        frame_np = orig_frame[:, :, ::-1]
        
        # Inference with tracking
        results = model.track(frame_np, conf=0.6, persist=True, tracker='bytetrack_custom.yaml')

        annotated_frame = frame_np.copy()
        cv2.rectangle(annotated_frame, (area_for_segment[0], area_for_segment[1]),
                (area_for_segment[2], area_for_segment[3]), (255, 0, 0), 2)

        potato_boxes = []
        potato_img_boxes = []
        _, orig_h, orig_w = orig_frame.shape 

        resized_size = 640
        scale_x = 1 #orig_w / resized_size
        scale_y = 1 #orig_h / resized_size

        if results and results[0].boxes is not None:
            for box in results[0].boxes:
                if box.id == None:
                    continue
                coords = box.xyxy[0].cpu().numpy().astype(int)
                x1, y1, x2, y2 = coords
                x1 = int(x1 * scale_x)
                x2 = int(x2 * scale_x)
                y1 = int(y1 * scale_y)
                y2 = int(y2 * scale_y)


                track_id = int(box.id.cpu().numpy()[0])
                track_set.add(track_id)

                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

                prev_size = map_track_size.get(track_id, (0, 0))
                cv2.putText(annotated_frame, f"ID: {track_id}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
                cv2.putText(annotated_frame, f"{round(prev_size[0], 2)}cm {round(prev_size[1], 2)}cm",
                            (x1, y1 + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

                if (area_for_segment[0] <= x1 <= area_for_segment[2] and
                    area_for_segment[1] <= y1 <= area_for_segment[3] and
                    area_for_segment[0] <= x2 <= area_for_segment[2] and
                    area_for_segment[1] <= y2 <= area_for_segment[3] and
                    prev_size[0] == 0 and prev_size[1] == 0):

                    img_box = frame_np[y1:y2, x1:x2]
                    potato_img_boxes.append(img_box)
                    potato_boxes.append([x1, y1, x2, y2, track_id])
                
                counter += 1

                data = {
                    "camera_id": camera_id,
                    "potato_id": track_id,  # Unique track ID
                    "type": "potato",
                    "size": f"{round(prev_size[0], 2)}cm {round(prev_size[1], 2)}cm",
                    "coordinates": [x1, y1, x2, y2],
                    "frame_id": frame_count,  # Frame ID
                    "passed": "yes",  # Example status
                    "sorted": "no"  # Example status
                }

                # Store the data in the dictionary using the potato_id as the key
                potato_data[track_id] = data

        # Segmentation
        if potato_img_boxes:
            results_seg = model_seg.predict(potato_img_boxes, imgsz=320)
            for i, result in enumerate(results_seg):
                x1, y1, x2, y2, track_id = potato_boxes[i]
                prev_major, prev_minor = map_track_size.get(track_id, (0, 0))

                if result.masks:
                    for mask in result.masks.xy:
                        abs_coords = mask + np.array([x1, y1])
                        abs_coords = abs_coords.astype(np.int32)

                        contour = np.int32([abs_coords]).reshape((-1, 1, 2))

                        if len(contour) >= 5:
                            ellipse = cv2.fitEllipse(contour)
                            _, axes, _ = ellipse
                            major_axis = max(axes) * cm_per_pixel
                            minor_axis = min(axes) * cm_per_pixel

                            avg_major = (major_axis + prev_major) / 2 if prev_major else major_axis
                            avg_minor = (minor_axis + prev_minor) / 2 if prev_minor else minor_axis

                            map_track_size[track_id] = (avg_major, avg_minor)

                            cv2.fillPoly(annotated_frame, [contour], colors)
        Count = 0 if len(track_set) == 0 else max(track_set)
        cv2.putText(annotated_frame, f"Potato Count: {Count}",
                    (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
        
        out.write(annotated_frame)

        potato_img_boxes = []
        potato_boxes = []
        img_box = None
        annotated_frame = None

out.release()

In [ ]:
results

In [ ]:
results[0].boxes

In [ ]:
track_set

In [ ]:
box